In [1]:
#this was initially a .py file instead of a jupyter notebook, 
#so the entire contents of the .py file has just been pasted into this chunk so that the code can be submitted as .html

import pandas as pd
import numpy as np

In [3]:
#MSN -> Movies and Shows (Netflix only)
MSN = pd.read_csv('Netflix Movies and TV Shows.csv')
#columns: show_id, type, title, director, cast, country, date_added, release_year, rating, duration, listed_in, description

#Sho -> TV shows
Sho = pd.read_csv('TV shows on Netflix, Prime Video, Hulu and Disney+.csv')
#columns: Unnamed: 0, Title, Year, Age, IMDb, Rotten Tomatoes, Netflix, Hulu, Prime Video, Disney+, type

#Mov -> Movies
Mov = pd.read_csv('Movies on Netflix, Prime Video, Hulu and Disney+.csv')
#columns: Unnamed: 0, Title, Year, Age, IMDb, Rotten Tomatoes, Netflix, Hulu, Prime Video, Disney+, Type, Directors, Genres, Country, Language, Runtime

In [9]:
#Making MSN's column names the same as Sho and Mov's
MSN.columns = ['show_id', 'type', 'Title', 'Directors', 'Cast', 'Country', 'date_added', 'Year', 'Flixable Rating', 'Runtime', 'Genres', 'description']

#fixing Mov's Type column name
Mov.rename(columns={'Type':'type'}, inplace=True)

#Making Sho and Mov's 'type' column match MSN's
def typeCleaner(row):
    if row['type'] == 0:
        return 'Movie'
    elif row['type'] == 1:
        return 'TV Show'
    else:
        #thankfully, this else never actually triggers, meaning Mov and Sho each only contain movies and shows, no strange extra formats (eg 'documentary', 'short film', etc)
        print(row)
        return row['type']

#Applying the typeCleaner function to 'type' column in Sho and Mov dataset.
Sho['type'] = Sho.apply(lambda row: typeCleaner(row), axis=1)
Mov['type'] = Mov.apply(lambda row: typeCleaner(row), axis=1)

In [10]:
#making a csv of Mov and Sho
Mov.to_csv('Mov.csv')
Sho.to_csv('Sho.csv')

In [13]:
#combining Mov and Sho

#MTV = Movies and TV shows
MTV = Mov.append(Sho)
#columns: Unnamed: 0, Title, Year, Age, IMDb, Rotten Tomatoes, Netflix, Hulu, Prime Video, Disney+, Type, Directors, Genres, Country, Language, Runtime

#Droping unnecessary columns
MTV = MTV.drop(columns=['Unnamed: 0'])
MSN = MSN.drop(columns=['show_id'])

In [18]:
#combing MSN and MTV into one final dataset
#FDF = Final DataFrame
FDF = MSN.merge(MTV, how='outer', on='Title')
# The columns this made were a huge mess.  I did my best to line up everything, but there were conflicting values.
# If you just want to drop all the rows with conflicting values, drop every row that has a string with ' or ' in it
# if you want to keep the values from MSN when there's a conflict, split on ' or ' and take the first value in the list that returns
# if you want to keep the values from MTV when there's a conflict, split on ' or ' and take the second value from the list that returns


In [20]:
#combining type columns
def typeCombiner(row):
    #if str(row['type_x']).strip == str(row['type_y']).strip:
    if row['type_x'] == row['type_y']:
        return row['type_x']
    elif str(row['type_x']) == 'nan':
        return row['type_y']
    elif str(row['type_y']) == 'nan':
        return row['type_x']
    else:
        #print(str(row['Title']) + ', ' + str(row['type_x']) + ', ' + str(row['type_y']))
        return str(row['type_x']) + ' or ' + str(row['type_y'])

FDF['type'] = FDF.apply(lambda row: typeCombiner(row), axis=1)
FDF = FDF.drop(columns=['type_x', 'type_y'])

In [22]:
#cleaning runtime columns
def runtime_xCleaner(row):
    if str(row['Runtime_x']) == 'nan':
        return row['Runtime_x']
    #elif str(row['Runtime_x']).find('Season') != -1:
    elif 'Season' in str(row['Runtime_x']):
        #print(str(row['Runtime_x']))
        return row['Runtime_x']
    else:
        #print(str(row['Runtime_x']))
        X = [int(i) for i in str(row['Runtime_x']).split() if i.isdigit()][0]
        #print(X)
        return str(X)

def runtime_yCleaner(row):
    if str(row['Runtime_y']) == 'nan':
        return row['Runtime_y']
    else:
        Y = int(row['Runtime_y'])
        #print(Y)
        return str(Y)

FDF['Runtime_x'] = FDF.apply(lambda row: runtime_xCleaner(row), axis=1)
FDF['Runtime_y'] = FDF.apply(lambda row: runtime_yCleaner(row), axis=1)

#combining runtime columns
def runtimeCombiner(row):
    if row['Runtime_x'] == row['Runtime_y']:
        return row['Runtime_x']
    elif str(row['Runtime_x']) == 'nan':
        return row['Runtime_y']
    elif str(row['Runtime_y']) == 'nan':
        return row['Runtime_x']
    else:
        #print(str(row['Runtime_x']) + ' or ' + str(row['Runtime_y']))
        return str(row['Runtime_x']) + ' or ' + str(row['Runtime_y'])

FDF['Runtime'] = FDF.apply(lambda row: runtimeCombiner(row), axis=1)
FDF = FDF.drop(columns=['Runtime_x', 'Runtime_y'])


In [23]:
#combining directors columns
from difflib import SequenceMatcher
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def directorsCombiner(row):
    if row['Directors_x'] == row['Directors_y']:
        return row['Directors_x']
    elif str(row['Directors_x']) == 'nan':
        return row['Directors_y']
    elif str(row['Directors_y']) == 'nan':
        return row['Directors_x']
    #this small block checks how similar the two column values are, and if they're somewhat similar it takes the longer of the two as the correct value
    elif similarity(str(row['Directors_x']), str(row['Directors_y'])) >= 0.33:
        #print('close enough')
        if len(str(row['Directors_y'])) > len(str(row['Directors_x'])):
            return row['Directors_y']
        else:
            return row['Directors_x']
    else:
        #print('no dice')
        #print(str(row['Directors_x']) + ' or ' + str(row['Directors_y']))
        return str(row['Directors_x']) + ' or ' + str(row['Directors_y'])

FDF['Directors'] = FDF.apply(lambda row: directorsCombiner(row), axis=1)
FDF = FDF.drop(columns=['Directors_x', 'Directors_y'])

In [24]:
#combining country columns
def countryCombiner(row):
    if row['Country_x'] == row['Country_y']:
        return row['Country_x']
    elif str(row['Country_x']) == 'nan':
        return row['Country_y']
    elif str(row['Country_y']) == 'nan':
        return row['Country_x']
    #this small block checks how similar the two column values are, and if they're somewhat similar it takes the longer of the two as the correct value
    elif similarity(str(row['Country_x']), str(row['Country_y'])) >= 0.33:
        if len(str(row['Country_y'])) > len(str(row['Country_x'])):
            return row['Country_y']
        else:
            return row['Country_x']
    else:
        return str(row['Country_x']) + ' or ' + str(row['Country_y'])

FDF['Country'] = FDF.apply(lambda row: countryCombiner(row), axis=1)
FDF = FDF.drop(columns=['Country_x', 'Country_y'])

In [25]:
#cleaning year columns
def year_xCleaner(row):
    if str(row['Year_x']) == 'nan':
        return row['Year_x']
    else:
        X = int(row['Year_x'])
        #print(X)
        return str(X)

def year_yCleaner(row):
    if str(row['Year_y']) == 'nan':
        return row['Year_y']
    else:
        Y = int(row['Year_y'])
        #print(Y)
        return str(Y)

FDF['Year_x'] = FDF.apply(lambda row: year_xCleaner(row), axis=1)
FDF['Year_y'] = FDF.apply(lambda row: year_yCleaner(row), axis=1)

#combining year columns
def yearCombiner(row):
    if str(row['Year_x']) == str(row['Year_y']):
        return row['Year_x']
    elif str(row['Year_x']) == 'nan':
        return row['Year_y']
    elif str(row['Year_y']) == 'nan':
        return row['Year_x']
    else:
        return str(row['Year_x']) + ' or ' + str(row['Year_y'])

FDF['Year'] = FDF.apply(lambda row: yearCombiner(row), axis=1)
FDF = FDF.drop(columns=['Year_x', 'Year_y'])

In [26]:
FDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25652 entries, 0 to 25651
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            25652 non-null  object 
 1   Cast             7085 non-null   object 
 2   date_added       7794 non-null   object 
 3   Flixable Rating  7797 non-null   object 
 4   Genres_x         7804 non-null   object 
 5   description      7804 non-null   object 
 6   Age              10519 non-null  object 
 7   IMDb             20623 non-null  float64
 8   Rotten Tomatoes  6169 non-null   object 
 9   Netflix          22355 non-null  float64
 10  Hulu             22355 non-null  float64
 11  Prime Video      22355 non-null  float64
 12  Disney+          22355 non-null  float64
 13  Genres_y         16469 non-null  object 
 14  Language         16145 non-null  object 
 15  type             25652 non-null  object 
 16  Runtime          21072 non-null  object 
 17  Directors   

In [29]:
#From the final dataset, 
#Drop unnecessary columns and keep only columns needed for this project
FDF = FDF.drop(columns=['Cast','Flixable Rating','Genres_x','Genres_y'])
FDF.columns

Index(['Title', 'date_added', 'description', 'Age', 'IMDb', 'Rotten Tomatoes',
       'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Language', 'type',
       'Runtime', 'Directors', 'Country', 'Year'],
      dtype='object')

In [ ]:
#making csv for FDF
FDF.to_csv("FDF.csv")